---
execute:
  cache: false
  eval: true
  echo: true
  warning: false
---

# Kriging (Gaussian Process Regression)

::: {.callout-note}
### Note
* This section is based on chapter 2.4 in @Forr08a.
* The following Python packages are imported:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import (array, zeros, power, ones, exp, multiply,
                    eye, linspace, spacing, sqrt, arange,
                    append, ravel)
from numpy.linalg import cholesky, solve
from scipy.spatial.distance import squareform, pdist, cdist

:::

<!-- bart21mSlides2022Lec-05 -->

##  From Gaussian RBF to Kriging Basis Functions

Kriging can be explained using the concept of radial basis functions (RBFs), which were introduced in @sec-rbf.
An RBF is a real-valued function whose value depends only on the distance from a certain point, called the center, usually in a multidimensional space.
The basis function is a function of the distance between the point $\vec{x}$ and the center $\vec{x}^{(i)}$. Other names for basis functions are *kernel* or *covariance* functions.


::: {#def-kriging-basis-function}

### The Kriging Basis Functions

Kriging (also known as Gaussian Process Regression) uses $k$-dimensional basis functions of the form 
$$
\psi^{(i)}(\vec{x}) = 
\psi(\vec{x}^{(i)}, \vec{x}) = \exp \left( - \sum_{j=1}^k \theta_j | x_{j}^{(i)} - x_{j} | ^{p_j} \right),
$$ {#eq-krigingbase}
where $\vec{x}$ and $\vec{x}^{(i)}$  denote the $k$-dim vector $\vec{x}= (x_1, \ldots, x_k)^T$ and $\vec{x}^{(i)}= (x_1^{(i)}, \ldots, x_k^{(i)})^T$, respectively.

:::

Kriging  uses a specialized basis function that offers greater flexibility than standard RBFs. Examining @eq-krigingbase, we can observe how Kriging builds upon and extends the Gaussian basis concept. The key enhancements of Kriging over Gaussian RBF can be summarized as follows:

* Dimension-specific width parameters: While a Gaussian RBF uses a single width parameter $1/\sigma^2$, Kriging employs a vector $\vec{\theta} = (\theta_1, \theta_2, \ldots, \theta_k)^T$. This allows the model to automatically adjust its sensitivity to each input dimension, effectively performing automatic feature relevance determination.
* Flexible smoothness control: The Gaussian RBF fixes the exponent at 2, producing uniformly smooth functions. In contrast, Kriging's dimension-specific exponents $\vec{p} = (p_1, p_2, \ldots, p_k)^T$ (typically with $p_j \in [1, 2]$) enable precise control over smoothness properties in each dimension.
* Unifying framework: When all exponents are set to $p_j = 2$ and all width parameters are equal ($\theta_j = 1/\sigma^2$ for all $j$), the Kriging basis function reduces exactly to the Gaussian RBF. This makes Gaussian RBF a special case within the more general Kriging framework.

These enhancements make Kriging particularly well-suited for engineering problems where variables may operate at different scales or exhibit varying degrees of smoothness across dimensions.
For now, we will only consider Kriging interpolation. We will cover Kriging regression later.


## Building the Kriging Model

Consider sample data $X$ and $\vec{y}$ from $n$ locations that are available in matrix form:
$X$ is a $(n \times k)$ matrix, where $k$ denotes the problem dimension and
$\vec{y}$ is a $(n\times 1)$ vector.
We want to find an expression for a predicted values at a new point $\vec{x}$, denoted as $\hat{y}$.

We start with an abstract, not really intuitive concept:
The observed responses $\vec{y}$ are considered as if they are from a stochastic
process, which will be denoted as
$$
\begin{pmatrix}
Y(\vec{x}^{(1)})\\
\vdots\\
Y(\vec{x}^{(n)})\\
\end{pmatrix}.
$$ {#eq-yvec-51}

The set of random vectors from @eq-yvec-51 (also referred to as a *random field*) has a mean of $\vec{1} \mu$, which is a $(n\times 1)$ vector.
The random vectors are correlated with each other using the basis function expression from @eq-krigingbase:
$$
\text{cor} \left(Y(\vec{x}^{(i)}),Y(\vec{x}^{(l)}) \right) = \exp\left(- \sum_{j=1}^k \theta_j |x_j^{(i)} - x_j^{(l)} |^{p_j}\right).
$$ {#eq-corr-kriging-51}
Using @eq-corr-kriging-51, we can compute the $(n \times n)$ correlation matrix $\Psi$ of the observed sample data as shown in @eq-corr-matrix-kriging-51, 

$$
\Psi = \begin{pmatrix}
\text{cor}\left(
Y(\vec{x}^{(1)}),
Y(\vec{x}^{(1)}) 
\right) & \ldots &
\text{cor}\left(
Y(\vec{x}^{(1)}),
Y(\vec{x}^{(n)}) 
\right)\\
\vdots  & \vdots &  \vdots\\
 \text{cor}\left(
Y(\vec{x}^{(n)}),
Y(\vec{x}^{(1)}) 
\right)&
\ldots &
\text{cor}\left(
Y(\vec{x}^{(n)}),
Y(\vec{x}^{(n)}) 
\right)
\end{pmatrix},
$$ {#eq-corr-matrix-kriging-51}

and a covariance matrix as shown in @eq-cov-matrix-kriging-52,

$$
\text{Cov}(Y, Y ) = \sigma^2\Psi.
$$ {#eq-cov-matrix-kriging-52}

This assumed correlation between the sample data reflects our expectation that an engineering function will behave in a certain way and it will be smoothly and continuous.

We now have a set of $n$ random variables ($\mathbf{Y}$) that are correlated with each other as described in the $(n \times n)$ correlation matrix
$\Psi$,
see @eq-corr-matrix-kriging-51.
The correlations depend on the absolute distances in dimension $j$ between the $i$-th and the $l$-th sample point $|x_j^{(i)} - x_j^{(l)}|$ and the 
corresponding parameters $p_j$ and $\theta_j$ for dimension $j$.
The correlation is intuitive, because when 

* two points move close together, then $|x_j^{(i)} - x_j| \to 0$ and $\exp \left(-|x_j^{(i)} - x_j|^{p_j} \right) \to 1$ (these points show very close correlation and $Y(x_j^{(i)}) = Y(x_j)$).
* two points move far apart, then $|x_j^{(i)} - x_j| \to \infty$ and $\exp \left(-|x_j^{(i)} - x_j|^{p_j} \right) \to 0$ (these points show very low correlation).

::: {#exm-kriging-corr-1}
### Correlations for different $p_j$

Three different correlations are shown in @fig-pval12: $p_j= 0.1, 1, 2$. The smoothness parameter $p_j$ affects the correlation:

* With $p_j=0.1$, there is basicaly no immediate correlation between the points and there is a near discontinuity between the points $Y(\vec{x}_j^{(i)})$ and $Y(\vec{x}_j)$.
* With $p_j=2$, the correlation is more smooth and we have a continuous gradient through $x_j^{(i)} - x_j$. 

Reducing $p_j$ increases the rate at which the correlation initially drops with distance. This is shown in @fig-pval12.


In [ ]:
#| echo: false
def squared_euclidean_distance(point1, point2, theta=1.0, p_val=2):
    return theta* (point1 - point2)**p_val

def inverse_exp_squared_distance(point1, point2, theta, p_val=2):
    return np.exp(-squared_euclidean_distance(point1=point1, point2=point2, theta=theta, p_val=p_val))

def generate_line(distance, step=0.01):
    return np.arange(0, distance+step, step)

def visualize_inverse_exp_squared_distance_theta(distance, point, theta_values, p_val=2):
    line = generate_line(distance)
    for theta in theta_values:
        distances = [inverse_exp_squared_distance(p, point, theta, p_val) for p in line]
        plt.plot(line, distances, label=f'theta={theta}')
    plt.legend()
    plt.grid()
    plt.show()

def visualize_inverse_exp_squared_distance_p(distance, point, theta, p_values):
    line = generate_line(distance)
    for p_val in p_values:
        distances = [inverse_exp_squared_distance(p, point, theta, p_val) for p in line]
        plt.plot(line, distances, label=f'p={p_val}')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
#| label: fig-pval12
#| fig-cap: "Correlations with varying $p$. $\\theta$ set to 1."
#| echo: false
visualize_inverse_exp_squared_distance_p(distance= 2.5, point=0, theta=1, p_values=[0.1, 1, 2.0])

:::

::: {#exm-kriging-corr-2}
### Correlations for different $\theta$

@fig-theta12 visualizes the correlation between two points $Y(\vec{x}_j^{(i)})$ and $Y(\vec{x}_j)$ for different values of $\theta$. The parameter $\theta$ can be seen as a *width* parameter:

* low $\theta_j$ means that all points will have a high correlation, with $Y(x_j)$ being similar across the sample.
* high $\theta_j$ means that there is a significant difference between the $Y(x_j)$'s.
* $\theta_j$ is a measure of how *active* the function we are approximating is.
* High $\theta_j$ indicate important parameters, see @fig-theta12.

In [ ]:
#| label: fig-theta12
#| fig-cap: "Correlations with varying $\\theta$. $p$ set to 2."
#| echo: false
visualize_inverse_exp_squared_distance_theta(distance= 2.5, point=0, theta_values=[0.1, 1, 10.0])

:::

Considering the activity parameter $\theta$ is useful in high-dimensional problems where it is difficult to visualize the design landscape and the effect of the variable is unknown. By examining the elements of the vector $\vec{\theta}$, we can identify the most important variables and focus on them. This is a crucial step in the optimization process, as it allows us to reduce the dimensionality of the problem and focus on the most important variables.



::: {#exm-corr-matrix-detailed}

#### The Correlation Matrix (Detailed Computation)

Let $n=4$ and $k=3$. The sample plan is represented by the following matrix $X$: 
$$
X = \begin{pmatrix} x_{11} & x_{12} & x_{13}\\
x_{21} & x_{22} & x_{23}\\
x_{31} & x_{32} & x_{33}\\
x_{41} & x_{42} & x_{43}\\ 
\end{pmatrix}
$$

To compute the elements of the matrix $\Psi$, the following $k$ (one for each of the $k$ dimensions) $(n,n)$-matrices have to be computed:

* For $k=1$, i.e., the first column of $X$:
$$
D_1 = \begin{pmatrix} x_{11} - x_{11} & x_{11} - x_{21} & x_{11} -x_{31} & x_{11} - x_{41} \\  x_{21} - x_{11} & x_{21} - x_{21} & x_{21} -x_{31} & x_{21} - x_{41} \\ x_{31} - x_{11} & x_{31} - x_{21} & x_{31} -x_{31} & x_{31} - x_{41} \\ x_{41} - x_{11} & x_{41} - x_{21} & x_{41} -x_{31} & x_{41} - x_{41} \\
\end{pmatrix}
$$

* For $k=2$, i.e., the second column of $X$:
$$
D_2 = \begin{pmatrix} x_{12} - x_{12} & x_{12} - x_{22} & x_{12} -x_{32} & x_{12} - x_{42} \\  x_{22} - x_{12} & x_{22} - x_{22} & x_{22} -x_{32} & x_{22} - x_{42} \\ x_{32} - x_{12} & x_{32} - x_{22} & x_{32} -x_{32} & x_{32} - x_{42} \\ x_{42} - x_{12} & x_{42} - x_{22} & x_{42} -x_{32} & x_{42} - x_{42} \\
\end{pmatrix}
$$

* For $k=3$, i.e., the third column of $X$:
$$
D_3 = \begin{pmatrix} x_{13} - x_{13} & x_{13} - x_{23} & x_{13} -x_{33} & x_{13} - x_{43} \\  x_{23} - x_{13} & x_{23} - x_{23} & x_{23} -x_{33} & x_{23} - x_{43} \\ x_{33} - x_{13} & x_{33} - x_{23} & x_{33} -x_{33} & x_{33} - x_{43} \\ x_{43} - x_{13} & x_{43} - x_{23} & x_{43} -x_{33} & x_{43} - x_{43} \\\end{pmatrix}
$$

Since the matrices are symmetric and the main diagonals are zero, it is sufficient to compute the following matrices:
$$
D_1 = \begin{pmatrix} 0 & x_{11} - x_{21} & x_{11} -x_{31} & x_{11} - x_{41} \\  0 &  0 & x_{21} -x_{31} & x_{21} - x_{41} \\ 0 & 0 & 0 & x_{31} - x_{41} \\ 0 & 0 & 0 & 0 \\\end{pmatrix}
$$
$$
D_2 = \begin{pmatrix} 0 & x_{12} - x_{22} & x_{12} -x_{32} & x_{12} - x_{42} \\  0 & 0 & x_{22} -x_{32} & x_{22} - x_{42} \\ 0 & 0 & 0 & x_{32} - x_{42} \\ 0 & 0 & 0 & 0 \\
\end{pmatrix}
$$

$$
D_3 = \begin{pmatrix} 0 & x_{13} - x_{23} & x_{13} -x_{33} & x_{13} - x_{43} \\  0 & 0 & x_{23} -x_{33} & x_{23} - x_{43} \\ 0 & 0 & 0 & x_{33} - x_{43} \\ 0 & 0 & 0 & 0 \\\end{pmatrix}
$$

We will consider $p_l=2$. The differences will be squared and multiplied by $\theta_i$, i.e.:

$$
D_1 = \theta_1 \begin{pmatrix} 0 & (x_{11} - x_{21})^2 & (x_{11} -x_{31})^2 & (x_{11} - x_{41})^2 \\  0 &  0 & (x_{21} -x_{31})^2 & (x_{21} - x_{41})^2 \\ 0 & 0 & 0 & (x_{31} - x_{41})^2 \\ 0 & 0 & 0 & 0 \\\end{pmatrix}
$$

$$
D_2 = \theta_2 \begin{pmatrix} 0 & (x_{12} - x_{22})^2 & (x_{12} -x_{32})^2 & (x_{12} - x_{42})^2 \\  0 & 0 & (x_{22} -x_{32})^2 & (x_{22} - x_{42})^2 \\ 0 & 0 & 0 & (x_{32} - x_{42})^2 \\ 0 & 0 & 0 & 0 \\\end{pmatrix}
$$

$$
D_3 = \theta_3 \begin{pmatrix} 0 & (x_{13} - x_{23})^2 & (x_{13} -x_{33})^2 & (x_{13} - x_{43})^2 \\  0 & 0 & (x_{23} -x_{33})^2 & (x_{23} - x_{43})^2 \\ 0 & 0 & 0 & (x_{33} - x_{43})^2 \\ 0 & 0 & 0 & 0 \\\end{pmatrix}
$$

The sum of the three matrices $D=D_1+ D_2 + D_3$ will be calculated next: 

$$
\begin{pmatrix} 0 & 
\theta_1  (x_{11} - x_{21})^2 + \theta_2 (x_{12} - x_{22})^2 + \theta_3  (x_{13} - x_{23})^2  &
\theta_1 (x_{11} -x_{31})^2 + \theta_2  (x_{12} -x_{32})^2 + \theta_3  (x_{13} -x_{33})^2 &
\theta_1  (x_{11} - x_{41})^2 + \theta_2  (x_{12} - x_{42})^2 + \theta_3 (x_{13} - x_{43})^2
\\  0 &  0 & 
\theta_1  (x_{21} -x_{31})^2 + \theta_2 (x_{22} -x_{32})^2 + \theta_3  (x_{23} -x_{33})^2 &
\theta_1  x_{21} - x_{41})^2 + \theta_2  (x_{22} - x_{42})^2 + \theta_3 (x_{23} - x_{43})^2
\\ 0 & 0 & 0 & 
\theta_1 (x_{31} - x_{41})^2 + \theta_2 (x_{32} - x_{42})^2 + \theta_3 (x_{33} - x_{43})^2
\\ 0 & 0 & 0 & 0 \\\end{pmatrix}
$$

Finally, $$ \Psi = \exp(-D)$$ is computed.

Next, we will demonstrate how this computation can be implemented in Python.
We will consider four points in three dimensions and compute the correlation matrix $\Psi$ using the basis function from @eq-krigingbase.
These points are placed at the origin, at the unit vectors, and at the points $(100, 100, 100)$ and $(101, 100, 100)$.
So, they form two clusters: one at the origin and one at $(100, 100, 100)$.


In [ ]:
theta = np.array([1,2,3])
X = np.array([ [1,0,0], [0,1,0], [100, 100, 100], [101, 100, 100]])
X

In [ ]:
def build_Psi(X, theta):
    n = X.shape[0]
    k = X.shape[1]
    D = zeros((k, n, n))
    for l in range(k):
        for i in range(n):
            for j in range(i, n):
                D[l, i, j] = theta[l]*(X[i,l] - X[j,l])**2
    D = sum(D)
    D = D + D.T
    return exp(-D)  

In [ ]:
Psi = build_Psi(X, theta)
Psi

In [ ]:
#| label: fig-corr-matrix-build_Psi
#| fig-cap: "Correlation matrix $\\Psi$."
#| echo: false
plt.imshow(Psi, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

:::


::: {#exm-corr-matrix-existing}
### Example: The Correlation Matrix (Using Existing Functions)

The same result as computed in the previous example can be obtained with existing python functions, e.g., from the package `scipy`.

In [ ]:
def build_Psi(X, theta, eps=sqrt(spacing(1))):
    return exp(- squareform(pdist(X,
                            metric='sqeuclidean',
                            out=None,
                            w=theta))) +  multiply(eye(X.shape[0]),
                                                   eps)

Psi = build_Psi(X, theta, eps=.0)
Psi

The condition number of the correlation matrix $\Psi$ is a measure of how well the matrix can be inverted. A high condition number indicates that the matrix is close to singular, which can lead to numerical instability in computations involving the inverse of the matrix, see @sec-conditon-number.

In [ ]:
np.linalg.cond(Psi)

:::

## MLE to estimate $\theta$ and $p$

### The Concentrated Log-Likelihood

Until now, the observed data $\vec{y}$ was not used.
We know what the correlations mean, but how do we estimate the values of $\theta_j$ and where does our observed data $y$ come in?
To estimate the values of $\vec{\theta}$ and $\vec{p}$, they are chosen to maximize the likelihood of $\vec{y}$,
which can be expressed in terms of the sample data 
$$
L\left(Y(\vec{x}^{(1)}), \ldots, Y(\vec{x}^{(n)}) | \mu, \sigma \right) = \frac{1}{(2\pi \sigma)^{n/2} |\vec{\Psi}|^{1/2}} \exp\left[ - \frac{(\vec{y} - \vec{1}\mu)^T \vec{\Psi}^{-1}(\vec{y} - \vec{1}\mu) }{2 \sigma^2}\right],
$$
and formulated as the log-likelihood:
$$
\ln(L) = - \frac{n}{2} \ln(2\pi \sigma) - \frac{1}{2} \ln |\vec{\Psi}| \frac{-(\vec{y} - \vec{1}\mu)^T \vec{\Psi}^{-1}(\vec{y} - \vec{1}\mu) }{2 \sigma^2}.
$$ {#eq-loglikelihood-55}

Optimization of the log-likelihood by taking derivatives with respect to $\mu$ and $\sigma$ results in
$$
\hat{\mu} = \frac{\vec{1}^T \vec{\Psi}^{-1} \vec{y}^T}{\vec{1}^T \vec{\Psi}^{-1} \vec{1}^T}
$$ {#eq-muhat-55}
and 
$$
\hat{\sigma}^2 = \frac{(\vec{y} - \vec{1}\mu)^T \vec{\Psi}^{-1}(\vec{y} - \vec{1}\mu)}{n}.
$$ {#eq-sigmahat-55}

Combining the equations, i.e., substituting @eq-muhat-55 and @eq-sigmahat-55 into @eq-loglikelihood-55 leads to the concentrated log-likelihood function: 
$$
\ln(L) \approx - \frac{n}{2} \ln(\hat{\sigma}) - \frac{1}{2} \ln |\vec{\Psi}|.
$$ {#eq-concentrated-loglikelihood}

::: {#rem-concentrated-loglikelihood}
### The Concentrated Log-Likelihood
* The first term in @eq-concentrated-loglikelihood requires information about the measured point (observations) $y_i$.
* To maximize $\ln(L)$, optimal values of $\vec{\theta}$ and $\vec{p}$ are determined numerically, because the function (@eq-concentrated-loglikelihood) is not differentiable.
:::

The concentrated log-likelihood function is very quick to compute. We do not need a statistical model, because we are only interested in the maximum likelihood estimate (MLE) of $\theta$ and $p$.
Optimizers such as Nelder-Mead, Conjugate Gradient, or Simulated Annealing can be used to determine optimal values for $\theta$ and $p$.
After the optimization, the correlation matrix $\Psi$ is build with the optimized $\theta$ and $p$ values. This is best (most likely) Kriging model for the given data $y$. 

Observing @fig-theta12, there's significant change between $\theta = 0.1$ and $\theta = 1$, just as there is between $\theta = 1$ and $\theta = 10$. Hence, it is sensible to search for $\theta$ on a logarithmic scale. Suitable search bounds typically range from $10^{-3}$ to $10^2$, although this is not a stringent requirement. Importantly, the scaling of the observed data does not affect the
values of $\hat{\theta}$, but the scaling of the design space does.
Therefore, it is advisable to consistently scale variable ranges between zero and one to ensure consistency in the degree of activity $\hat{\theta}_j$ represents across different problems.


### Implementing an MLE of the Model Parameters

The matrix algebra necessary for calculating the likelihood is the most computationally intensive aspect of the Kriging process. It's crucial to ensure that the code implementation is as efficient as possible.

Given that $\Psi$ (our correlation matrix) is symmetric, only half of the matrix needs to be computed before adding it to its transpose. When calculating the log-likelihood, several matrix inversions are required. The fastest approach is to conduct one Cholesky factorization and then apply backward and forward substitution for each inverse.

The Cholesky factorization is applicable only to positive-definite matrices, which $\Psi$ generally is. However, if $\Psi$ becomes nearly singular, such as when the $\mathbf{x}^{(i)}$'s are densely packed, the Cholesky factorization might fail. In these cases, one could employ an LU-decomposition, though the result might be unreliable. When $\Psi$ is near singular, the best course of action is to either use regression techniques or, as we do here, assign a poor likelihood value to parameters generating the near singular matrix, thus diverting the MLE search towards better-conditioned $\Psi$ matrices.

When working with correlation matrices, increasing the values on the main diagonal of a matrix will increase the absolute value of its determinant. A critical numerical consideration in calculating the concentrated log-likelihood is that for poorly conditioned matrices, $\det(\Psi)$ approaches zero, leading to potential numerical instability. To address this issue, it is advisable to calculate $\ln(\lvert\Psi\rvert)$ in @eq-concentrated-loglikelihood using twice the sum of the logarithms of the diagonal elements of the Cholesky factorization. This approach provides a more numerically stable method for computing the log-determinant, as the Cholesky decomposition $\Psi = L L^T$ allows us to express $\ln(\lvert\Psi\rvert) = 2\sum_{i=1}^{n} \ln(L_{ii})$, avoiding the direct computation of potentially very small determinant values.


## Kriging Prediction

We will use the Kriging correlation $\Psi$ to predict new values based on the observed data. The matrix algebra involved for calculating the likelihood is the most computationally intensive part of the Kriging process. Care must be taken that the computer code is as efficient as possible.

Basic elements of the Kriging based surrogate optimization 
such as interpolation, expected improvement, and regression are presented. The presentation follows the approach described in @Forr08a and @bart21i.

Main idea for prediction is that the new $Y(\vec{x})$ should be consistent with the old sample data $X$.  For a new prediction $\hat{y}$ at $\vec{x}$, the value of $\hat{y}$ is chosen so that it maximizes the likelihood of the sample data $X$ and the prediction, given the (optimized) correlation parameter $\vec{\theta}$ and $\vec{p}$ from above.  The observed data $\vec{y}$ is augmented with the new prediction $\hat{y}$ which results in the augmented vector $\vec{\tilde{y}} = ( \vec{y}^T, \hat{y})^T$. A vector of correlations between the observed data and the new prediction is defined as

$$ \vec{\psi} = \begin{pmatrix}
\text{cor}\left(
Y(\vec{x}^{(1)}),
Y(\vec{x}) 
\right) \\
\vdots  \\
\text{cor}\left(
Y(\vec{x}^{(n)}),
Y(\vec{x}) 
\right)
\end{pmatrix}
=
\begin{pmatrix}
\vec{\psi}^{(1)}\\
\vdots\\
\vec{\psi}^{(n)}
\end{pmatrix}.
$$

::: {#def-augmented-correlation-matrix}
### The Augmented Correlation Matrix

The augmented correlation matrix is constructed as
$$ \tilde{\vec{\Psi}} =
\begin{pmatrix}
\vec{\Psi} & \vec{\psi} \\
\vec{\psi}^T & 1
\end{pmatrix}.
$$
::: 

The log-likelihood of the augmented data is
$$
\ln(L) = - \frac{n}{2} \ln(2\pi) - \frac{n}{2} \ln(\hat{\sigma}^2) - \frac{1}{2} \ln |\vec{\hat{\Psi}}| -  \frac{(\vec{\tilde{y}} - \vec{1}\hat{\mu})^T \vec{\tilde{\Psi}}^{-1}(\vec{\tilde{y}} - \vec{1}\hat{\mu})}{2 \hat{\sigma}^2},
$$ {#eq-loglikelihood-augmented}

where $\vec{1}$ is a vector of ones and $\hat{\mu}$ and $\hat{\sigma}^2$ are the MLEs from @eq-muhat-55 and @eq-sigmahat-55. Only the last term in  @eq-loglikelihood-augmented depends on $\hat{y}$, so we need only consider this term in the maximization. Details cen be found in @Forr08a. Finally, the MLE for $\hat{y}$ can be calculated as
$$
\hat{y}(\vec{x}) = \hat{\mu} + \vec{\psi}^T \vec{\tilde{\Psi}}^{-1} (\vec{y} - \vec{1}\hat{\mu}).
$$ {#eq-mle-yhat}


@eq-mle-yhat reveals two important properties of the Kriging predictor:

* Basis functions: The basis function impacts the vector $\vec{\psi}$, which contains the $n$ correlations between the new point $\vec{x}$ and the observed locations. Values from the $n$ basis functions are added to a mean base term $\mu$ with weightings
$$
\vec{w} = \vec{\tilde{\Psi}}^{(-1)} (\vec{y} - \vec{1}\hat{\mu}).
$$
* Interpolation: The predictions interpolate the sample data. When calculating the prediction at the $i$th sample point, $\vec{x}^{(i)}$, the $i$th column of $\vec{\Psi}^{-1}$ is $\vec{\psi}$, and $\vec{\psi}  \vec{\Psi}^{-1}$ is the $i$th unit vector. Hence,

$$
\hat{y}(\vec{x}^{(i)}) = y^{(i)}.
$$


## Kriging Example: Sinusoid Function 

Toy example in 1d where the response is a simple sinusoid measured at eight equally spaced  $x$-locations in the span of a single period of oscillation.

### Calculating the Correlation Matrix $\Psi$

The correlation matrix $\Psi$ is based on the pairwise squared distances between the input locations. Here we will use $n=8$ sample locations and $\theta$ is set to 1.0.

In [ ]:
n = 8
X = np.linspace(0, 2*np.pi, n, endpoint=False).reshape(-1,1)
print(np.round(X, 2))

Evaluate at sample points

In [ ]:
y = np.sin(X)
print(np.round(y, 2))

We have the data points shown in @tbl-sin-data.

| $x$ | $y$ |
|---:|--:|
| 0.0 | 0.0 |
| 0.79 | 0.71 |
| 1.57 | 1.0 |
| 2.36 | 0.71 |
| 3.14 | 0.0 |
| 3.93 | -0.71 |
| 4.71 | -1.0 |
| 5.5 | -0.71 |
: Data points for the sinusoid function {#tbl-sin-data}

The data points are visualized in @fig-sin-data.

In [ ]:
#| label: fig-sin-data
#| fig-cap: "Sin(x) evaluated at 8 points."
plt.plot(X, y, "bo")
plt.title(f"Sin(x) evaluated at {n} points")
plt.grid()
plt.show()

### Computing the $\Psi$ Matrix

We will use the `build_Psi` function from @exm-corr-matrix-existing to compute the correlation matrix $\Psi$.
$\theta$ should be an array of one value, because we are only working in one dimension ($k=1$).

In [ ]:
theta = np.array([1.0])
Psi = build_Psi(X, theta)
print(np.round(Psi, 2))

@fig-sin-corr visualizes the $(8, 8)$ correlation matrix $\Psi$.

In [ ]:
#| label: fig-sin-corr
#| fig-cap: "Correlation matrix $\\Psi$ for the sinusoid function."
#| echo: false
plt.imshow(Psi, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

### Selecting the New Locations

We would like to predict at $m = 100$ new locations (or testign locations)
in the interval $[0, 2\pi]$. The new locations are stored in the variable `x`.

In [ ]:
m = 100
x = np.linspace(0, 2*np.pi, m, endpoint=False).reshape(-1,1)

### Computing the $\psi$ Vector

Distances between testing locations $x$ and training data locations $X$.

In [ ]:
def build_psi(X, x, theta, eps=sqrt(spacing(1))):
    n = X.shape[0]
    k = X.shape[1]
    m = x.shape[0]
    psi = zeros((n, m))
    theta = theta * ones(k)
    D = zeros((n, m))
    D = cdist(x.reshape(-1, k),
              X.reshape(-1, k),
              metric='sqeuclidean',
              out=None,
              w=theta)    
    psi = exp(-D)
    # return psi transpose to be consistent with the literature
    print(f"Dimensions of psi: {psi.T.shape}")
    return(psi.T)

psi = build_psi(X, x, theta)

@fig-sin-corr-pred visualizes the $(8, 100)$ prediction matrix $\psi$.

In [ ]:
#| label: fig-sin-corr-pred
#| fig-cap: "Visualization of the predition matrix $\\psi$"
#| echo: false
plt.imshow(psi, cmap='hot', interpolation='nearest')
# plt.colorbar()
plt.show()

### Predicting at New Locations

Computation of the predictive equations.

In [ ]:
U = cholesky(Psi).T
one = np.ones(n).reshape(-1,1)
mu = (one.T.dot(solve(U, solve(U.T, y)))) / one.T.dot(solve(U, solve(U.T, one)))
f = mu * ones(m).reshape(-1,1) + psi.T.dot(solve(U, solve(U.T, y - one * mu)))
print(f"Dimensions of f: {f.shape}")

To compute $f$, @eq-mle-yhat is used. 

### Visualization

In [ ]:
plt.plot(x, f, color = "orange", label="Fitted")
plt.plot(x, np.sin(x), color = "grey", label="Original")
plt.plot(X, y, "bo", label="Measurements")
plt.title("Kriging prediction of sin(x) with {} points.\n theta: {}".format(n, theta[0]))
plt.legend(loc='upper right')
plt.show()

## Jupyter Notebook

:::{.callout-note}

* The Jupyter-Notebook of this lecture is available on GitHub in the [Hyperparameter-Tuning-Cookbook Repository](https://github.com/sequential-parameter-optimization/Hyperparameter-Tuning-Cookbook/blob/main/006_num_gp.ipynb)

:::
